# Buổi 9: Nhóm và sắp xếp dữ liệu
- Biến đổi dữ liệu
- Nhóm và sắp xếp dữ liệu
- Xử lý các vấn đề với dữ liệu: thiếu dữ liệu, sai kiểu dữ liệu, trùng lặp dữ liệu, ...
- Thực hành

In [65]:
import pandas as pd

def clean_networth(networth): # loại bỏ phần tử đầu - cuối: $ B
    return networth[1:-1]
    

df = pd.read_csv('./Billionaires.csv', index_col=[0])
df.info() # lấy thông tin dữ liệu

df['Net Worth'] = df['Net Worth'].apply(clean_networth)
df['Net Worth'] = pd.to_numeric(df['Net Worth']) # chuyển từ string => Float

## Biến đổi dữ liệu cho cột age
df['Age'] = df['Age'].astype('Int64')
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 2640 entries, 1 to 2540
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Name                 2640 non-null   object 
 1   Net Worth            2640 non-null   object 
 2   Age                  2576 non-null   float64
 3   Country | Territory  2640 non-null   object 
 4   Source               2640 non-null   object 
 5   Industry             2640 non-null   object 
dtypes: float64(1), object(5)
memory usage: 144.4+ KB


,Name,Net Worth,Age,Country | Territory,Source,Industry
Rank,,,,,,
1,Bernard Arnault & family,211.0,74,France,LVMH,Fashion & Retail
2,Elon Musk,180.0,51,United States,"Tesla, SpaceX",Automotive
3,Jeff Bezos,114.0,59,United States,Amazon,Technology
4,Larry Ellison,107.0,78,United States,Oracle,Technology
5,Warren Buffett,106.0,92,United States,Berkshire Hathaway,Finance & Investments


Biến đổi dữ liệu cột 'Net Worth' từ string => float

Tổng tài sản lớn nhất của các tỉ phú nằm ở những lĩnh vực nào?

In [66]:
df_industry = df[['Industry', 'Net Worth']].groupby('Industry').sum() # trả về đối tượng DF Groupby
# Sx theo thứ tự 
df_industry = df_industry.sort_values(by='Net Worth', ascending=False) # từ lớn => nhỏ (ngược lại là true)
df_industry


,Net Worth
Industry,
Technology,1871.0
Fashion & Retail,1698.8
Finance & Investments,1605.1
Manufacturing,1025.9
Food & Beverage,957.2
Diversified,905.2
Real Estate,657.4
Healthcare,643.2
Automotive,525.3


Thực hành: Tính trung bình tài sản các tỷ phú theo châu lục
- Chưa có cột châu lục?
- Cần biến đổi từ cột: Country | Territory
- Sử dụng thư viện và hàm hỗ trợ để tạo cột "Continent"

In [67]:
import pycountry_convert as pc

def get_continent(country_name):
    try:
        country_code = pc.country_name_to_country_alpha2(country_name)
        continent_code = pc.country_alpha2_to_continent_code(country_code)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except:
        print(country_name)
        return 'unknown'
    

df['Continent'] = df['Country | Territory'].apply(get_continent)

## Chỉ show 2 cột cần thôi
df[['Net Worth', 'Continent']].groupby("Continent").mean()

## Nếu muốn tìm ra cái không xác định
# df[df['Continent'] == 'unknown']

Eswatini (Swaziland)


,Net Worth
Continent,
Africa,4.421053
Asia,3.547064
Europe,4.853279
North America,6.025276
Oceania,3.924000
South America,3.375000
unknown,6.500000


# 4. Làm sạch dữ liệu

In [68]:
df = pd.read_csv("./hockey_players_data.csv")
# df.head(10)
# df.tail()

# Tìm các giá trị bị khuyết (NaN) và đếm
df.isnull().sum()


number           0
name             0
position         0
games_played     0
goals           10
assists          0
PIM              0
birthdate        0
age              0
birthplace       2
weight          11
height           1
dtype: int64

In [69]:
# Xử lý giá trị bị khuyết
# df.dropna() (Hạn chế dùng thôi)


###### Drop các cột không cần thiết
df.drop(['birthdate', 'birthplace', 'weight'], axis=1, inplace=True)
df


#### Lọc hàng và kiểm tra được cầu thủ nào bị thiếu 'goals'
df[df.goals.isna()]


,number,name,position,games_played,goals,assists,PIM,age,height
4,67,Emma Woods,F,6,NaN,4,0,24,5.04
6,27,Amy Curlew,F,6,NaN,2,4,23,5.07
10,27,Shiann Darkangelo,F,6,NaN,3,0,22,5.08
15,25,Emma Greco,D,6,NaN,0,0,22,5.08
16,15,Natalie Marcuzzi,F,6,NaN,0,0,27,5.08
17,91,Jenna McParland,F,6,NaN,0,0,33,5.08
18,91,Henna McParland,M,5,NaN,0,0,33,5.08
19,27,Kristen Barbara,D,6,NaN,0,0,26,5.10
20,29,Julie Allen,F,6,NaN,0,0,26,5.07
21,34,Elaine Chuli,G,6,NaN,0,0,24,5.11


In [70]:
## Thay thế các giá trị NaN thành một giá trị mặc định bằng: 
df.goals = df.goals.fillna(0)
df.tail()

,number,name,position,games_played,goals,assists,PIM,age,height
17,91,Jenna McParland,F,6,0.0,0,0,33,5.08
18,91,Henna McParland,M,5,0.0,0,0,33,5.08
19,27,Kristen Barbara,D,6,0.0,0,0,26,5.10
20,29,Julie Allen,F,6,0.0,0,0,26,5.07
21,34,Elaine Chuli,G,6,0.0,0,0,24,5.11


In [71]:
### Kiểm tra lại kiểu dữ liệu các cột đang là gì?
# df.dtypes

df.age = df.age.astype("int") # thay đổi từ float => int
## Do chiều cao đang là dạng chuỗi => ép kiểu về float
df.height = pd.to_numeric(df.height)


df.dtypes

number            int64
name             object
position         object
games_played      int64
goals           float64
assists           int64
PIM               int64
age               int64
height          float64
dtype: object

In [72]:
### Xử lý hàng bị trùng
df.duplicated().sum() # 1


## Loại bỏ hàng bị trùng
df = df.drop_duplicates()
df.duplicated().sum() # 0

np.int64(0)

In [74]:
# ### Đối với một số hàng chỉ bị trùng "số áo" kết hợp để xử lý như sau
df[df.number.duplicated()]

df.drop_duplicates(subset=['number'])


,number,name,position,games_played,goals,assists,PIM,age,height
0,13,Mikyla Grant-Mentis,F,6,5.0,4,4,22,5.08
1,44,Lindsay Eastwood,D,6,4.0,5,10,23,5.08
2,2,Taylor Woods,F,6,4.0,4,4,26,5.04
3,19,Brooke Boquist,F,6,2.0,2,2,24,5.03
4,67,Emma Woods,F,6,0.0,4,0,24,5.04
5,21,Breanne Wilson-Bennett,F,6,2.0,4,2,24,5.03
6,27,Amy Curlew,F,6,0.0,2,4,23,5.07
8,18,Taytum Clairmont,F,6,1.0,3,21,26,5.11
11,7,Mackenzie MacNeil,F,6,2.0,1,1,24,5.08
12,12,Emily Fluke,F,6,1.0,4,4,27,5.07
